In [1]:
import numpy as np
import pandas as pd
import re
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
import nltk

In [2]:
# Downloading necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Loading SpaCy model for text preprocessing
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Reading data from SMSSpamCollection
def read_data(path):
    labels = []
    texts = []
    with open('SMSSpamCollection') as f:
        lines = f.readlines()
        labels = [line.split('\t')[0] for line in lines]
        texts = [line.split('\t')[1].strip() for line in lines]  # Strip to remove extra newline
    return labels, texts


# Preprocessing the data using re, NLTK and SpaCY
def preprocess_text(text):
    # Converting all to lower case
    text = text.lower()

    # Remove single characters and digits
    text = re.sub(r'\b\w\b', '', text)  # Remove single characters
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize using NLTK
    words = word_tokenize(text)

    # Remove stopwords and lemmatize using SpaCy
    processed_words = [token.lemma_ for token in nlp(' '.join(words)) if token.text not in stop_words and len(token.text) > 1]

    preprocessed_text = ' '.join(processed_words)

    return preprocessed_text

# Changing labels ham/spam to 0/1
def encode_labels(labels):
    for i in range(len(labels)):
        if labels[i] == 'ham':
            labels[i] = 0
        else:
            labels[i] = 1
    return labels



In [4]:
# Find the most frequent spam words in X_train
def find_frequent_spam_words(texts, labels, num_words=10):
    spam_messages = [texts[i] for i in range(len(labels)) if labels[i] == 1]

    # Initialize a defaultdict to track unique message counts for each word
    word_to_count = defaultdict(int)

    for message in spam_messages:
        unique_words = set(message.split())
        for word in unique_words:
            word_to_count[word] += 1  # Increment the count for each unique word

    most_common_spam_words = sorted(word_to_count.items(), key=lambda item: item[1], reverse=True)[:num_words]
    freq_spam_words = [word for word, _ in most_common_spam_words]
    return freq_spam_words

# Function to calculate average word length
def calculate_avg_word_length(text):
    words = text.split()
    return np.mean([len(word) for word in words]) if words else 0

# Extract features from texts, using the most common spam words dynamically
def extract_features(texts, frequent_spam_words):
    length_of_message = [len(text) for text in texts]
    word_count = [len(text.split()) for text in texts]
    num_frequent_spam_words = [sum(1 for word in text.split() if word in frequent_spam_words) for text in texts]
    avg_word_length = [calculate_avg_word_length(text) for text in texts]

    features = pd.DataFrame({
        'length_of_message': length_of_message,
        'word_count': word_count,
        'num_frequent_spam_words': num_frequent_spam_words,
        'avg_word_length': avg_word_length,
    })

    return features

In [5]:
import numpy as np

# Calculate entropy
def entropy(y):
    p1 = np.mean(y)
    if p1 == 0 or p1 == 1:  # Avoid log(0) by returning entropy 0 directly when pure
        return 0
    return -p1 * np.log2(p1) - (1 - p1) * np.log2(1 - p1)

# Information gain based on entropy
def information_gain(X, y, feature_column):
    parent_entropy = entropy(y)
    median_value = np.median(X[:, feature_column])
    left_mask = X[:, feature_column] < median_value
    right_mask = X[:, feature_column] >= median_value

    # If a split results in no data in one of the branches, return zero information gain
    if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
        return 0

    left_entropy = entropy(y[left_mask])
    right_entropy = entropy(y[right_mask])

    left_weight = len(y[left_mask]) / len(y)
    right_weight = len(y[right_mask]) / len(y)

    weighted_entropy = left_weight * left_entropy + right_weight * right_entropy
    info_gain = parent_entropy - weighted_entropy

    return info_gain

# Decision tree node for classification
class ClassificationNode:
    def __init__(self):
        self.label = None
        self.feature = None
        self.threshold = None
        self.left = None
        self.right = None

# Decision Tree Classifier using CART
class CARTClassifier:
    def __init__(self, max_depth=5, min_samples=10):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.root = None

    def fit(self, features, target):
        self.root = self.build_tree(features, target, depth=0)

    # Building the decision tree
    def build_tree(self, features, target, depth):
        cur_node = ClassificationNode()
        no_of_samples = features.shape[0]

        if len(np.unique(target)) == 1 or no_of_samples < self.min_samples or depth >= self.max_depth:
            cur_node.label = 1 if np.mean(target) >= 0.5 else 0  # Majority class
            return cur_node

        split_feature_dict = self.get_best_feature(features, target)
        if split_feature_dict is None:
            cur_node.label = 1 if np.mean(target) >= 0.5 else 0
            return cur_node

        feature_ind = split_feature_dict['feature_ind']
        threshold = split_feature_dict['threshold']

        left_mask = features[:, feature_ind] < threshold
        right_mask = features[:, feature_ind] >= threshold

        features_left, target_left = features[left_mask], target[left_mask]
        features_right, target_right = features[right_mask], target[right_mask]

        cur_node.feature = feature_ind
        cur_node.threshold = threshold
        cur_node.left = self.build_tree(features_left, target_left, depth + 1)
        cur_node.right = self.build_tree(features_right, target_right, depth + 1)

        return cur_node

    # Calculating the best feature
    def get_best_feature(self, features, target):
        best_info_gain = -float('inf')
        best_feature = None
        best_threshold = None

        for feature_ind in range(features.shape[1]):
            info_gain = information_gain(features, target, feature_ind)

            if info_gain > best_info_gain:
                best_info_gain = info_gain
                best_feature = feature_ind
                best_threshold = np.median(features[:, feature_ind])

        if best_feature is None:
            return None

        return {'feature_ind': best_feature, 'threshold': best_threshold}

    def predict(self, X):
        predictions = np.array([self._predict_single(x, self.root) for x in X])
        return predictions

    def _predict_single(self, x, node):
        if node.label is not None:
            return node.label
        if x[node.feature] < node.threshold:
            return self._predict_single(x, node.left)
        else:
            return self._predict_single(x, node.right)


In [6]:
# Reading the data
labels, texts = read_data('SMSSpamCollection')

# Preprocess
texts = [preprocess_text(text) for text in texts]

# Encode labels: 1->Spam, 0->Ham
labels = encode_labels(labels)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Find the most frequent spam words in X_train
frequent_spam_words = find_frequent_spam_words(X_train, y_train, num_words=41)
print(f"Most frequent spam words: {frequent_spam_words}\n")

# Extract features from the training and test sets
train_features = extract_features(X_train, frequent_spam_words)
test_features = extract_features(X_test, frequent_spam_words)
print("Total Feature Extracted:", train_features.shape[1])
print("Extracted Features are:",)
for i, feature_index in enumerate(train_features.columns):
    print(f"Feature {i + 1}: {feature_index}")

Most frequent spam words: ['call', 'free', 'txt', 'ur', 'text', 'mobile', 'claim', 'reply', 'stop', 'get', 'send', 'prize', 'new', 'service', 'cash', 'win', 'award', 'urgent', 'contact', 'phone', 'week', 'please', 'tone', 'box', 'min', 'guarantee', 'customer', 'nokia', 'per', 'ppm', 'chat', 'message', 'receive', 'line', 'voucher', 'draw', 'day', 'go', 'po', 'today', 'try']

Total Feature Extracted: 4
Extracted Features are:
Feature 1: length_of_message
Feature 2: word_count
Feature 3: num_frequent_spam_words
Feature 4: avg_word_length


In [7]:
train_features.head()

,length_of_message,word_count,num_frequent_spam_words,avg_word_length
0,96,17,1,4.705882
1,26,5,1,4.400000
2,19,4,0,4.000000
3,53,10,0,4.400000
4,96,17,2,4.705882


In [8]:
test_features.head()

,length_of_message,word_count,num_frequent_spam_words,avg_word_length
0,18,3,0,5.333333
1,107,23,2,3.695652
2,20,4,0,4.250000
3,36,6,1,5.166667
4,12,3,1,3.333333


In [9]:
train_features.describe()

,length_of_message,word_count,num_frequent_spam_words,avg_word_length
count,4459.000000,4459.000000,4459.000000,4459.000000
mean,46.893250,8.257233,1.129850,4.759298
std,36.634283,6.064961,1.746741,1.204708
min,0.000000,0.000000,0.000000,0.000000
25%,20.000000,4.000000,0.000000,4.000000
50%,35.000000,6.000000,1.000000,4.666667
75%,70.000000,12.000000,1.000000,5.269697
max,478.000000,76.000000,11.000000,37.000000


In [10]:
test_features.describe()

,length_of_message,word_count,num_frequent_spam_words,avg_word_length
count,1115.000000,1115.000000,1115.000000,1115.000000
mean,47.898655,8.401794,1.216143,4.785278
std,36.217644,6.052730,1.866484,1.086208
min,0.000000,0.000000,0.000000,0.000000
25%,21.000000,4.000000,0.000000,4.142857
50%,37.000000,7.000000,1.000000,4.666667
75%,69.000000,12.000000,2.000000,5.303846
max,304.000000,60.000000,11.000000,11.000000


In [11]:
# Convert labels to NumPy arrays for further processing
y_train = np.array(y_train)
y_test = np.array(y_test)

# Selecting to 3 features based on information gain
info_gains = [information_gain(train_features.values, y_train, i) for i in range(train_features.shape[1])]

#Showing information gain for each features
print("Information Gain for each feature:")
for i, gain in enumerate(info_gains):
    print(f"{train_features.columns[i]}: {gain:.4f}")

top_3_features = np.argsort(info_gains)[-3:]  # Get indices of top 3 features
print("\nTop 3 Features based on Information Gain:")
for i, feature_index in enumerate(top_3_features):
    print(f"Feature {i + 1}: {train_features.columns[feature_index]}")

# Train a decision tree using only the top 4 features
X_train_top3 = train_features.values[:, top_3_features]
X_test_top3 = test_features.values[:, top_3_features]

# Train the CART classifier
tree_classifier = CARTClassifier(max_depth=5, min_samples=10)
tree_classifier.fit(X_train_top3, y_train)

# Accuracy score on train set
y_train_pred = tree_classifier.predict(X_train_top3)
accuracy_train = accuracy_score(y_train, y_train_pred)

# Predict on the test set
y_pred = tree_classifier.predict(X_test_top3)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nClassification Accuracy on the train set using custom decision tree model: {accuracy_train:.4f}")
print(f"\nClassification Accuracy on the test set using custom decision tree model: {accuracy:.4f}")


Information Gain for each feature:
length_of_message: 0.1082
word_count: 0.0833
num_frequent_spam_words: 0.1077
avg_word_length: 0.0354

Top 3 Features based on Information Gain:
Feature 1: word_count
Feature 2: num_frequent_spam_words
Feature 3: length_of_message

Classification Accuracy on the train set using custom decision tree model: 0.9388

Classification Accuracy on the test set using custom decision tree model: 0.9309


In [12]:
# Using decision tree from sklearn to calculate accuracy
from sklearn.tree import DecisionTreeClassifier

sklearn_dtree = DecisionTreeClassifier(max_depth=5, min_samples_split=10)
sklearn_dtree.fit(X_train_top3, y_train)

# Accuracy score on train set
y_train_pred = tree_classifier.predict(X_train_top3)
accuracy_train = accuracy_score(y_train, y_train_pred)

# Predict on the test set
y_pred = sklearn_dtree.predict(X_test_top3)

accuracy = accuracy_score(y_test, y_pred)
print(f"\nClassification Accuracy on the train set using sklearn decision tree model: {accuracy_train:.4f}")
print(f"\nClassification Accuracy on the test set using sklearn decision tree model: {accuracy:.4f}")


Classification Accuracy on the train set using sklearn decision tree model: 0.9388

Classification Accuracy on the test set using sklearn decision tree model: 0.9372
